# Build Chains and Results  
Instructions: https://fsworks.atlassian.net/wiki/spaces/~558841686/pages/edit-v2/2087550977

In [2]:
import subprocess
instance_name = 'service_dev'
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = 'query_web_test'
tasks_types = 'tdas'
results = 'prt'
query = 'web' 
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
from run_service import service_manager
service_manager(instance_name, data_file_name, experiment, tasks_types, results, query)

EC2 instance service_dev i-0249408ea16bc730b started
The instance is running
Data file uploaded to S3
Calculation finished
EC2 instance stopped
Preparing results
results_bucket, bucket_prt_path: chainsresults query_web_test/prt
The following results files are available in the query_web_test directory:
 chains.parquet,results.xlsx,prt
12370 chains written to chains.parquet
200924 tasks rows written to parquet files in the prt sub-directory


# Query Results  

In [4]:
import duckdb
import os
con = duckdb.connect()
prt_files_path = os.path.join(experiment, 'prt')
results_files = "'{e}/*.parquet'".format(e=prt_files_path)
query = """SELECT * FROM {rf} LIMIT 10""".format(rf=results_files)
query_df = con.execute(query).df()
# To save the query results run:
#query_df.to_excel('query_df.xlsx')
display(query_df)

,ID,ChainID,TaskID,NeighbourID,Dependency,TaskType,Label,PlannedStart,PlannedEnd,ActualStart,ActualEnd,Float1,Status,File,planned_duration,actual_duration
0,MI20-EMS-N2-60000,C1,C1T1,MI20-EMS-N2-110100,FS,TT_FinMile,JS0 - T0 TF - NTP,2021-06-30,2021-06-30,2021-06-30,2021-06-30,0,TK_Complete,EMS_DCMA_DD_23.08.graphml,0,0.0
1,MI20-EMS-N2-110100,C1,C1T2,MI20-EMS-N2-110110,FS,TT_Task,Activits IDR System,2021-01-07,2021-01-07,2021-01-07,2022-03-17,0,TK_Complete,EMS_DCMA_DD_23.08.graphml,0,259.0
2,MI20-EMS-N2-110110,C1,C1T3,MI20-EMS-N2-61086,FS,TT_FinMile,OIM - IDR System,2022-03-17,2022-03-17,2022-03-17,2022-03-17,0,TK_Complete,EMS_DCMA_DD_23.08.graphml,0,0.0
3,MI20-EMS-N2-61086,C1,C1T4,MI20-EMS-N2-61090,FS,TT_FinMile,OIM - IDRs - Interface Gate [TRAIN CAF],2022-10-19,2022-10-19,nan,nan,41,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
4,MI20-EMS-N2-61090,C1,C1T5,MI20-EMS-N2-61110,FS,TT_Mile,IIM - IGR - Interface Gate - OFFICIEL,2022-10-27,2022-10-27,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
5,MI20-EMS-N2-61110,C1,C1T6,MI20-EMS-N2-61130,FS,TT_Mile,IIM - PGR - Preliminary Gate - OFFICIEL,2023-07-04,2023-07-04,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
6,MI20-EMS-N2-61130,C1,C1T7,MI20-EMS-N2-61150,FS,TT_Mile,IIM - CGR 1st - 1st Train Critical Gate - OFFICIEL,2023-07-07,2023-07-07,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
7,MI20-EMS-N2-61150,C1,C1T8,MI20-EMS-N2-121108,FS,TT_Mile,IIM - CGR Full - Full Critical Gate - OFFICIEL,2023-08-08,2023-08-08,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
8,MI20-EMS-N2-121108,C1,C1T9,MI20-EMS-N2-121138,FS,TT_Mile,IIM - CGR Full - Full Critical Gate [Train Design] - OFFICIEL,2023-08-08,2023-08-08,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
9,MI20-EMS-N2-121138,C1,C1T10,MI20-EMS-N2-121168,FS,TT_Mile,IIM - FEI - First Equipment Inspection [Train Design] - OFFICIEL,2024-04-15,2024-04-15,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan


# Read and Write Chains  

In [6]:
import pandas as pd
chains_path = os.path.join(experiment, 'chains.parquet')
chains_df = pd.read_parquet(chains_path)
chains = list(chains_df['Chain'])
chains = '\n'.join(chains)+'\n'
with open('chains.txt', 'w') as f: f.write(chains)
display(chains_df.head())
display(chains_df.info())

,Chain
0,MI20-EMS-N2-60000<>MI20-EMS-N2-110100<>MI20-EMS-N2-110110<>MI20-EMS-N2-61086<>MI20-EMS-N2-61090<...
1,MI20-EMS-N2-60000<>MI20-EMS-N2-122338<>MI20-EMS-N2-122348<>MI20-EMS-N2-122351<>MI20-EMS-N2-12235...
2,MI20-EMS-N2-60000<>MI20-EMS-N2-120131<>MI20-EMS-N2-120132<>MI20-EMS-N2-61060<>MI20-EMS-N2-61073<...
3,MI20-EMS-N2-60000<>MI20-EMS-N2-70055<>MI20-EMS-N2-601812<>MI20-EMS-N2-601862<>MI20-EMS-N2-601881...
4,MI20-EMS-N2-60000<>MI20-EMS-N2-510034<>MI20-EMS-N2-510036<>MI20-EMS-N2-61060<>MI20-EMS-N2-61073<...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12370 entries, 0 to 12369
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Chain   12370 non-null  object
dtypes: object(1)
memory usage: 96.8+ KB


None